In [1]:
%load_ext autoreload
%autoreload 2

from scribbles_creator import *
from PIL import Image
import napari
import numpy as np

## Create scribble annotation from an image mask of the cell pose dataset

In [42]:
def create_cellpose_scribble(folder_path, img_num, scribble_width=1, sk_goal_perc=0.05, sq_size=20, sq_pix_range=(10, 100), lines_goal_perc=0.05, line_pix_range=(10, 100), mode="all"):
    # Read the ground truth as an image
    image_path = f"{folder_path}/{str(img_num).zfill(3)}"
    ground_truth = np.array(Image.open(image_path + "_masks.png"))
    # Summarize all non-background classes into one class (we are testing semantic segmentation not instance segmentation)
    ground_truth[ground_truth>0] = 2
    # Set the background class to 1 (since the scribble annotation assumes class 0 to represent non-annotated pixels)
    ground_truth[ground_truth==0] = 1

    # Generate the scribble annotation for the ground truth
    scribbles = create_scribble(ground_truth, scribble_width=scribble_width, sk_goal_perc=sk_goal_perc, sq_size=sq_size, sq_pix_range=sq_pix_range, lines_goal_perc=lines_goal_perc, line_pix_range=line_pix_range, mode=mode)
    perc_labelled = np.sum(scribbles>0) / (scribbles.shape[0] * scribbles.shape[1]) * 100

    return scribbles, perc_labelled

In [50]:
folder_path = "./imgs/train"
im_num = 331
scribbles, perc_labelled = create_cellpose_scribble(folder_path, im_num, scribble_width=1, sk_goal_perc=0.05, sq_size=20, sq_pix_range=(40, 50), lines_goal_perc=0.05, line_pix_range=(10, 20), mode="all")
print(f"Percentage of labelled pixels: {perc_labelled:.2f}%")

class 1: 194 = 0.20%
   prim_sk_squares: 49 = 0.05%
   sec_sk_squares: 87 = 0.09%
   lines: 58 = 0.06%
class 2: 125 = 0.20%
   prim_sk_squares: 42 = 0.07%
   sec_sk_squares: 42 = 0.07%
   lines: 41 = 0.07%
Percentage of labelled pixels: 0.20%


## Save or show the scribbles and ground truth

In [48]:
# Show the image, ground truth and the scribble annotation in a napari viewer
v = napari.Viewer()
image_path = f"{folder_path}/{str(im_num).zfill(3)}"
ground_truth = np.array(Image.open(image_path + "_masks.png"))
v.add_labels(ground_truth)
v.add_labels(scribbles)
image = np.array(Image.open(image_path + "_img.png"))
# v.add_image(image)

In [ ]:
# Save the scribbles and ground truth image to a file (first, convert the numpy arrays to images)
scribbles_image = Image.fromarray(scribbles)
ground_truth_image = Image.fromarray(ground_truth)
scribbles_image.save(image_path + '_scribbles.png')
ground_truth_image.save(image_path + '_ground_truth.png')

## When we have performed segmentation in Napari, we can calculate its accuracy

In [ ]:
segmentation = v.layers['segmentation'].data
acc = np.round(100*np.sum(segmentation == ground_truth) / np.sum(ground_truth > 0), 2)
print(f"Accuracy (pixels correctly segmented): {acc}%")